In [3]:
from src.cluster_description import Cluster, ClustersDescription
import pyranges as pr
import numpy as np

_GMT_PATH = "./data/c5.go.bp.v7.5.1.entrez.gmt"
_FANTOM_PATH = "./data/FANTOM5_CAGE_peak_entrez_gene_tbl.tsv"

# Parses the GMT file returining a dict containing for each 
# Returns a dictionary mapping each gene entrez id to the ontology group it belongs to.
def parse_gmt(path: str) -> dict[str, list[str]]:
    # Create a dictionary to store the results
    results = {}

    # Open the GMT file
    with open(path, "r") as f:
        # For each line in the file
        for line in f:
            # Split the line into a list of words
            words = line.split("\t")

            # The motif name
            gene_set = words[0]
            link = words[1]
            genes = words[2:]
            # strip each gene
            genes = [gene.strip() for gene in genes]

            # Add the gene set to the dictionary
            results[gene_set] = [link, genes]

    # invert the dictionary
    gmt_inverse : dict[str, list[str]] = {}

    # For each gene set
    for gene_set, (link, genes) in results.items():
        for gene in genes:
            if gene not in gmt_inverse:
                gmt_inverse[gene] = []
            gmt_inverse[gene].append(gene_set)
        
    return gmt_inverse, list(results.keys())

# read the table from the path
def parse_fantom_data(path: str) -> dict[str, str]:
    # Create a dictionary to store the results
    results: dict[str, str] = {}

    header_found = False

    # Open the file
    with open(path, "r") as f:
        # For each line in the file
        for line in f:
            # Go to the next line if it starts with a #
            if line.startswith("#"):
                continue

            # deal with the header
            if not header_found:
                header_found = True
                continue

            # Split the line into a list of words
            words = line.split("\t")
            cage_id = words[0].strip()
            entrez_id = words[1].strip()

            # Add the gene to the dictionary
            results[cage_id] = entrez_id
        
    # Return the results
    return results

# Maps the given entry id to the gene entrez id
def overlaps_to_entrez(overlaps: list[str], peak_to_entrez : dict[str, str]) -> list[str]:
    entrez_ids = []
    for overlap in overlaps:
        entrez_ids.append(peak_to_entrez.get(overlap, None))
    
    # Remove duplicates and NAs
    entrez_ids = [entrez_id for entrez_id in entrez_ids if entrez_id not in ["NA", None]]
    return entrez_ids

# Return the ontology group to which each entrez id belongs to.
def map_entrez_to_ontology(entrez_ids: list[str], gene_ontology : dict[str, str]) -> list[str]:
    ontology_hist = {}
    unknown_entrez_ids = []
    for entrez_id in entrez_ids:
        if entrez_id not in gene_ontology:
            unknown_entrez_ids.append(entrez_id)
            continue
        ontology_groups = gene_ontology[entrez_id]

        for ontology_group in ontology_groups:
            if ontology_group not in ontology_hist:
                ontology_hist[ontology_group] = 0
            ontology_hist[ontology_group] += 1
    
    return ontology_hist, unknown_entrez_ids


### Maps the provided features (passed in as the list of names to the the given ontology groups) and returns an histogram of the counts of each group.
### Like this:
### {   
###     "GO:0005737": 1,
###     "GO:0005739": 1,
###     "GO:0005740": 3,
###     ...
### }
### The features are expected to be already void of duplicates as it would lead to a double counting.
def get_groups_for_features(features: list[str], peak_to_entrez: dict[str, str], gene_ontology : dict[str, str]) -> dict[str, int]:
    # Get the entrez ids
    entrez_ids = overlaps_to_entrez(features, peak_to_entrez)

    # Get the ontology
    groups, _ = map_entrez_to_ontology(entrez_ids, gene_ontology)
    
    # Return the groups
    return groups, len(entrez_ids)

# General Imports and Parsing:

In [4]:
# Reading the data.
peak_to_entrez = parse_fantom_data(_FANTOM_PATH)
gene_ontology, gene_sets = parse_gmt(_GMT_PATH)

# Getting the Background Data.
The gene sets counts are computed for all the features present in the data folder to produce a background distribution. 

In [5]:
# Getting the background
background = pr.read_bed(f"../data/features/HMEC/CAGE/features.bed")

# Getting the representation 
feature_names = background.Name.values.flatten()
expected_groups, population_size = get_groups_for_features(feature_names, peak_to_entrez, gene_ontology)

### Reading the Observed Data
The data about which gene set are enriched in the clusters is generated.
To do so, the clusters in analysis are read from the folder. We then find the clusters features in the feature file which overlap each cluster and see which of gene set are present in those features.

In [6]:
# Read the information about the cluster enriched in both CAGE and CTCF
CLUSTERS_IN_ANALYSIS_PATH = "../results/intersected.tsv"

def parse_enrichment_file(path : str):
    results = {}

    with open(path, "r") as f:
        for line in f:
            words = line.split("\t")
            chromosome = words[0].strip()
            clusters = [w.strip() for w in words[1:]]

            results[chromosome] = clusters

    return results

# Reading the clusters:
clusters_in_analysis = parse_enrichment_file(CLUSTERS_IN_ANALYSIS_PATH)

In [7]:
# Reading the features.
features = pr.read_bed("../data/features/HMEC/CAGE/features.bed")

# the list keeping track of which genes are present in the features in analysis.
observed_genes = []

# For each cluster
for chromo, clusters_names in clusters_in_analysis.items():
    # Reading the clusters.
    clusters = ClustersDescription(f"../data/clusters/HMEC/{chromo}_spec_res.json", chromo)
    
    # Looping over the cluster (strings)
    for cluster in clusters_names:

        # Getting the actual cluster object
        cluster = clusters[cluster]

        # Counting the overlaps
        overlaps = cluster.find_overlaps(chromo, features)
        overlaps_names = overlaps.Name.values.flatten()
        
        # Getting the observed genes
        observed_genes.extend(overlaps_to_entrez(overlaps_names, peak_to_entrez))

        # removing duplicates
        observed_genes = list(set(observed_genes))

# Getting an histogram of the observed gene groups.
observed_groups, _ = map_entrez_to_ontology(observed_genes, gene_ontology)

# Performing the hypergeometric test:
The hypergeometric test is performed on the background distribution and the observed distribution.

In [8]:
from scipy.stats import hypergeom
from tqdm import tqdm

# The p_values
p_values = {}

# Looping over all the gene set in the ontology
for _set in tqdm(gene_sets):
    
    # run the hypergeometric test
    M = population_size # population size
    n = expected_groups.get(_set, 0) # number of successes in the population
    N = len(observed_genes) # size of the sample
    x = observed_groups.get(_set, 0) # number of successes in the sample

    p_value = hypergeom.sf(x-1, M, n, N)
    p_values[_set] = [p_value, x, n]

100%|██████████| 7658/7658 [01:10<00:00, 108.53it/s]


In [9]:
# Convert the values to a dataframe
import pandas as pd
p_values_df = pd.DataFrame(p_values).T 

In [10]:
# Add the column names
p_values_df.columns = ["p_value", "observed", "background"]

In [11]:
# apply fdr correction
import statsmodels.stats.multitest as smm

#adding a column called q_value
p_values_df["q_value"] = smm.multipletests(p_values_df["p_value"], method="fdr_bh")[1]

In [13]:
# sort the dataframe on q_value
p_values_df.sort_values("q_value", inplace=True)

# Add index as a column
p_values_df["gene_set"] = p_values_df.index

# Make the gene_set the first column
p_values_df = p_values_df[["gene_set", "q_value", "p_value", "observed", "background"]]

In [190]:
#Save the first 100 results
p_values_df[:100].to_csv("../results/enrichment_results.tsv", sep="\t", index=False)